In [ ]:
# Binary classification on circular data using a simple neural network

# Setting GPU: Notebook settings -> T3 GPU (for faster training)

import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets

# Generate synthetic circular data (two concentric circles)
n_pts = 500
X, y = datasets.make_circles(n_samples=n_pts, random_state=123, noise=0.1, factor=0.2)


# Convert data to PyTorch tensors
x_data = torch.Tensor(X)
y_data = torch.Tensor(y.reshape(500, 1))  # Reshape to column vector

print(x_data.shape, y_data.shape)  # Check shape of input and label tensors


# Plot the data points by class
def scatter_plot():
    plt.scatter(X[y == 0, 0], X[y == 0, 1], color='red')
    plt.scatter(X[y == 1, 0], X[y == 1, 1], color='blue')

scatter_plot()  # Show the data distribution


# Define a simple neural network model
class Model(nn.Module):
    def __init__(self, input_size, H1, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, H1)  # First hidden layer
        self.linear2 = nn.Linear(H1, output_size)  # Output layer

    def forward(self, x):
        x = torch.sigmoid(self.linear(x))  # Apply sigmoid to first layer
        x = torch.sigmoid(self.linear2(x))  # Apply sigmoid to output layer
        return x

    def predict(self, x):
        return 1 if self.forward(x) >= 0.5 else 0  # Binary prediction


# Create model instance
model = Model(2, 4, 1)
print(list(model.parameters()))  # View model parameters


# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.1)  # Adam optimizer


# Function to calculate accuracy
def get_accuracy(y_pred, y_true):
    predicted = y_pred >= 0.5  # Convert probabilities to 0 or 1
    correct = predicted.eq(y_true.bool())  # Compare with true labels
    acc = correct.sum().item() / len(y_true)  # Compute accuracy
    return acc


# Training loop
epochs = 1000
losses = []
accuracies = []


for i in range(epochs):
    optimizer.zero_grad()  # Reset gradients from previous step
    y_pred = model(x_data)  # Forward pass (model prediction)
    loss = criterion(y_pred, y_data)  # Compute loss

    acc = get_accuracy(y_pred, y_data)  # Compute accuracy
    print(f"epoch: {i}, loss: {loss.item():.4f}, accuracy: {acc:.4f}")

    losses.append(loss.item())  # Store loss
    accuracies.append(acc)  # Store accuracy
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights


# Plot training loss over epochs
plt.figure()
plt.plot(range(epochs), losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Training Loss')


# Plot training accuracy over epochs
plt.figure()
plt.plot(range(epochs), accuracies)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Training Accuracy')


# Visualize decision boundary
def plot_decision_boundary(X, y):
    x_span = np.linspace(min(X[:, 0]), max(X[:, 0])) # X-axis range
    y_span = np.linspace(min(X[:, 1]), max(X[:, 1])) # Y-axis range
    xx, yy = np.meshgrid(x_span, y_span)
    grid = torch.Tensor(np.c_[xx.ravel(), yy.ravel()]) # Flatten grid for prediction
    pred_func = model(grid)
    z = pred_func.view(xx.shape).detach().numpy() # Reshape to match grid for plotting
    plt.contourf(xx, yy, z)


# Plot decision boundary and data points
plt.figure()
plot_decision_boundary(x_data, y_data)
scatter_plot() # Overlay original data points
plt.title('Decision Boundary')
plt.show()